In [1]:
import pandas as pd
import duckdb
import os
import os, json
from uuid import uuid4
import pandas as pd
import numpy as np
import pickle

In [2]:
works_df = pd.read_parquet("../data/grela_works_metadata.parquet")
works_df.head(5)

,grela_source,grela_id,author,title,not_before,not_after,lagt_tlg_epithet,lagt_genre,lagt_provenience,noscemus_place,noscemus_genre,noscemus_discipline,title_short,emlap_noscemus_id,place_publication,place_geonames,author_viaf,title_viaf,date_random
0,lagt,lagt_ggm0001.ggm001,Anonymous,Anametresis Pontou,1.0,400.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,372.0
1,lagt,lagt_ogl0001.ogl001,Pinytus,De Epistola Pinyti ad Dionysium,101.0,200.0,[],[],christian,None,None,None,None,NaN,None,None,NaN,NaN,135.0
2,lagt,lagt_pta0001.pta001,Severian of Gabala,De fide et lege naturae,400.0,409.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,403.0
3,lagt,lagt_pta0001.pta002,Severian of Gabala,De paenitentia et compunctione,400.0,409.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,409.0
4,lagt,lagt_pta0001.pta003,Severian of Gabala,In ascensionem domini nostri Iesu Christi et i...,400.0,409.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,402.0


In [3]:
works_df.sample(20)

,grela_source,grela_id,author,title,not_before,not_after,lagt_tlg_epithet,lagt_genre,lagt_provenience,noscemus_place,noscemus_genre,noscemus_discipline,title_short,emlap_noscemus_id,place_publication,place_geonames,author_viaf,title_viaf,date_random
1422,lagt,lagt_tlg0636.tlg001,Socrates Epistulae,Epistulae,-468.0,-398.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,-403.0
7546,cc,cc_7577,Prosper Aquitanus,Epigramma in obtractorem Augustini,390.0,463.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,391.0
272,lagt,lagt_tlg0007.tlg136,Plutarch,Περὶ Στωικῶν ἐνιγμάτων,96.0,120.0,['Biographi' 'Philosophici/-ae'],[],pagan,None,None,None,None,NaN,None,None,NaN,NaN,119.0
6974,cc,cc_7010,Auctor incertus,Epistola de fide,NaN,NaN,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN
8835,cc,cc_8853,Theodulfus Aurelianensisc.785-818,Carmina,785.0,818.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,793.0
5549,cc,cc_15572,Albertanus Brixiensis,Liber de doctrina dicendi et tacendi,1246.0,1246.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1246.0
3216,cc,cc_11029,Petrus diaconus,Descriptio sacri Casini montis,1007.0,1150.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1035.0
6844,cc,cc_6883,Auctores varii,Leges in Donatistas,NaN,NaN,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN
580,lagt,lagt_tlg0057.tlg006,Galen,De constitutione artis medicae ad Patrophilum,129.0,216.0,['Medici'],[],pagan,None,None,None,None,NaN,None,None,NaN,NaN,177.0
2598,cc,cc_10418,Auctor incertus,Monumenta de treuga Domini,NaN,NaN,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN


In [17]:
vulgate_works_df = pd.read_parquet("../data/vulgate_works_df.parquet")

# Save sentences_data
with open("../data/vulgate_sentences.pkl", "rb") as f:
    sentences_data = pickle.load(f)

In [13]:
vulgate_works_df.head(5)

,grela_id,title
0,vulgate_tlg0031.tlg001.obi-lat,Vulgate - Matthew
1,vulgate_tlg0031.tlg002.obi-lat,Vulgate - Mark
2,vulgate_tlg0031.tlg003.obi-lat,Vulgate - Luke
3,vulgate_tlg0031.tlg004.obi-lat,Vulgate - John
4,vulgate_tlg0031.tlg005.obi-lat,Vulgate - Acts


In [9]:
conn = duckdb.connect("/srv/data/grela_v0-2.duckdb")
#conn.execute("CREATE TABLE works AS SELECT * FROM works_df")

In [14]:
# Register the pandas DataFrame as a temporary DuckDB table
conn.register("vulgate_works_df", vulgate_works_df)

In [15]:
conn.execute("""
    INSERT INTO works (grela_source, grela_id, title)
    SELECT 'vulgate', grela_id, title FROM vulgate_works_df
""")

In [19]:
sentences_rows = []

for work_id, chapter_verse, verse_text, token_data in sentences_data():
    grela_id = f"vulgate_{work_id}"
    sentence_id = f"{grela_id}:{chapter_verse}"
    position = len([s for s in sentences_rows if s["grela_id"] == grela_id])  # per-work index

    sentences_rows.append({
        "sentence_id": sentence_id,
        "grela_id": grela_id,
        "position": position,
        "text": verse_text
    })

sentences_df = pd.DataFrame(sentences_rows)


In [23]:
len(sentences_data())

35254

In [21]:
len(sentences_df)

35254

In [24]:
conn.register("sentences_df", sentences_df)

conn.execute("""
    INSERT INTO sentences (sentence_id, grela_id, position, text)
    SELECT sentence_id, grela_id, position, text FROM sentences_df
""")

In [26]:
tokens_rows = []
token_counter = 0  # global unique token ID

for work_id, chapter_verse, verse_text, token_data in sentences_data():
    grela_id = f"vulgate_{work_id}"
    sentence_id = f"{grela_id}:{chapter_verse}"

    char_index = 0

    for token_text, lemma, pos, ref_string in token_data:
        start = char_index
        end = start + len(token_text)

        tokens_rows.append({
            "sentence_id": sentence_id,
            "grela_id": grela_id,
            "token_text": token_text,
            "lemma": lemma,
            "pos": pos,
            "char_start": start,
            "char_end": end,
            "token_id": token_counter
        })

        token_counter += 1
        char_index = end + 1
tokens_df = pd.DataFrame(tokens_rows)

In [27]:
len(tokens_df)

603091

In [28]:
conn.register("tokens_df", tokens_df)

conn.execute("""
    INSERT INTO tokens (
        sentence_id, grela_id, token_text, lemma, pos,
        char_start, char_end, token_id
    )
    SELECT
        sentence_id, grela_id, token_text, lemma, pos,
        char_start, char_end, token_id
    FROM tokens_df
""")

In [36]:
# let's update the token_id based on the actual rows...

conn.execute("""
    UPDATE tokens
    SET token_id = new_ids.rn
    FROM (
        SELECT rowid, ROW_NUMBER() OVER (ORDER BY sentence_id, char_start) AS rn
        FROM tokens
        WHERE grela_id LIKE 'vulgate_%'
    ) AS new_ids
    WHERE tokens.rowid = new_ids.rowid;
""")

In [37]:
conn.unregister("vulgate_works_df")
conn.unregister("sentences_df")
conn.unregister("tokens_df")

In [38]:
# Query to list all tables in the database
tables_query = "SHOW TABLES;"
tables = conn.execute(tables_query).fetchall()
tables

[('sentences',), ('tokens',), ('works',)]

In [39]:
for table in tables:
    print(conn.execute(f"SELECT COUNT(*) as row_count FROM {table[0]};").fetchdf())

   row_count
0   26602568
   row_count
0  381248237
   row_count
0      11117


<llm-snippet-file>database-building.ipynb</llm-snippet-file>


In [4]:
conn.close()

NameError: name 'conn' is not defined

In [9]:
# look
sents_emlap.sample(10)

,sentence_id,grela_id,position,text
110291,emlap_100051_3639,emlap_100051,3639,"Ad uenarum humores erassos, lentos, ac pituito..."
49304,emlap_100070_5892,emlap_100070,5892,"Figulina pinguis, sanguis draconis paucus, ter..."
131811,emlap_100061_612,emlap_100061,612,ibid. 91.
124958,emlap_100046_1953,emlap_100046,1953,"Uerum eiusmodi pascua quandoque peiora sunt, q..."
77197,emlap_100042_1078,emlap_100042,1078,"Nec est quod ita mireris, medicinam aliquam in..."
198900,emlap_100015_2627,emlap_100015,2627,"Prima aqua erit clara, & ualet podagrae humidae:"
95195,emlap_100032_1298,emlap_100032,1298,"quia ad formam oui dispositum est, & dicitur s..."
34064,emlap_100013_2815,emlap_100013,2815,"Omni mane & sero de hac aqua impones ad aures,..."
216215,emlap_100011_399,emlap_100011,399,Promissio & diuisio dicendorum de operationib.
94951,emlap_100032_1054,emlap_100032,1054,Hec est huius rei radix ut qui eam addiscere u...


In [ ]:
%%time
sents_nos, tokens_nos = process_sentences_from_dir(noscemus_sents_data_dir, "noscemus")

In [15]:
sents_nos.sample(10)

,sentence_id,grela_id,position,text
743250,noscemus_668522_21910,noscemus_668522,21910,Elacatena quoque salsamento idoneus piscis est...
2565541,noscemus_914304_9018,noscemus_914304,9018,pag. 212.
2661058,noscemus_928147_13394,noscemus_928147,13394,Secunda ibi.
3149822,noscemus_732628_8192,noscemus_732628,8192,Idem inquietis & stolidis ingemus euenit:
9961985,noscemus_901145_78964,noscemus_901145,78964,fiat confectio in rotulis ponderis. 3 8.
3466975,noscemus_631363_13372,noscemus_631363,13372,Calceolario Ueronensi Pharmacopol Ioannis Bapt...
4135971,noscemus_929375_5561,noscemus_929375,5561,Inter sinistrum humerum & Tertiam 2.
8949356,noscemus_906961_21555,noscemus_906961,21555,Harduini in 410.
8237679,noscemus_664561_4548,noscemus_664561,4548,"Pitem attenuat, resoluit, Matt."
2789886,noscemus_835557_24530,noscemus_835557,24530,b. 50.


In [16]:
tokens_nos.sample(10)

,sentence_id,grela_id,token_text,lemma,pos,char_start,char_end,page_idx,textblock_idx
95331451,noscemus_767766_1405,noscemus_767766,camelorum,camelus,NOUN,13,22,None,None
5493374,noscemus_704336_15313,noscemus_704336,impressit,imprimo,VERB,53,62,None,None
99530955,noscemus_918511_10291,noscemus_918511,In,in,ADP,0,2,None,None
91258461,noscemus_906964_7937,noscemus_906964,De,de,ADP,26,28,None,None
92824347,noscemus_655273_3785,noscemus_655273,circulos,circulus,NOUN,82,90,None,None
31477593,noscemus_756874_21222,noscemus_756874,headed,,VERB,35,41,None,None
108428896,noscemus_756878_20567,noscemus_756878,Les,Les,PROPN,0,3,None,None
100898942,noscemus_888138_57149,noscemus_888138,corporis,corpus,NOUN,38,46,None,None
115204905,noscemus_744031_4973,noscemus_744031,",",",",PUNCT,84,85,None,None
106678555,noscemus_900765_6687,noscemus_900765,a,ab,ADP,88,89,None,None


In [17]:
sents_lagt, tokens_lagt = process_sentences_from_dir(lagt_sents_data_dir, "lagt")

In [ ]:
sents_cc, tokens_cc = process_sentences_from_dir(cc_sents_data_dir, "cc")

In [ ]:
# Merge
all_sents = pd.concat([sents_cc, sents_nos, sents_emlap, sents_lagt])
all_tokens = pd.concat([tokens_cc, tokens_nos, tokens_emlap, tokens_lagt])

## Add tokens and sentences into the database

In [ ]:
# add table with sentence data
conn.execute("CREATE TABLE IF NOT EXISTS sentences AS SELECT * FROM all_sents")

In [ ]:
# Store in batches if necessary:
for i, chunk in enumerate(np.array_split(all_tokens, 100)):
    conn.register("chunk", chunk)
    conn.execute("INSERT INTO tokens SELECT * FROM chunk")

In [ ]:
lemmata_df = all_tokens.groupby(["lemma", "pos"]).size().reset_index(name="count")
conn.execute("CREATE TABLE lemmata AS SELECT * FROM lemmata_df")